In [1]:
import pandas as pd

In [2]:
# Cargamos los dataset
def read_datasets(datasets):
    dataframes = {}
    for name in datasets:
        dataframes[name] = pd.read_csv(f'/content/drive/MyDrive/data/{name}.csv', encoding='utf-8', lineterminator='\n')
    return dataframes

datasets = ['df_recommend','df_sentiment', 'df_user_genre']
dataframes = read_datasets(datasets)

# Convertimos a df para trabajar las funciones.
df_recommend = dataframes['df_recommend']
df_sentiment = dataframes['df_sentiment']
df_user_genre = dataframes['df_user_genre']

----
+ def **PlayTimeGenre( *`genero` : str* )**:
    Debe devolver `año` con mas horas jugadas para dicho género.
  
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [3]:
def PlayTimeGenre(genre: str):
  """
  Devuelve el año con más horas jugadas para un género dado.

  Args:
    genre: El género del juego.

  Returns:
    El año de lanzamiento con más horas jugadas para un género dado.
    Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}
  """

  # Filtra el DataFrame por género
  df_genre = df_user_genre[df_user_genre['genres'].str.contains(genre)]

  # Agrupa por año y calcula la suma total de horas jugadas para cada año
  playtime_by_year = df_genre.groupby('release_year')['playtime_forever'].sum()

  # Encuentra el año con más horas jugadas
  max_playtime_year = playtime_by_year.idxmax()
  response = {f"Año de lanzamiento con más horas jugadas para el género {genre}" : max_playtime_year}

  return response

In [4]:
PlayTimeGenre('Action')

{'Año de lanzamiento con más horas jugadas para el género Action': 2012}

---

+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
			     "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [5]:
def UserForGenre(genre: str):
  """
  Devuelve el usuario que acumula más horas jugadas para un género dado.

  Args:
    genre: El género del juego.

  Returns:
    El usuario con más horas jugadas y una lista de la acumulación de horas jugadas por año.
    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}
  """

  # Filtra el DataFrame por género
  df_genre = df_user_genre[df_user_genre["genres"].str.contains(genre)]

  # Agrupa por usuario y año, y calcula la suma total de horas jugadas para cada usuario y año
  playtime_by_user_year = df_genre.groupby(['user_id', 'release_year'])['playtime_forever'].sum().reset_index()

  # Encuentra el usuario con más horas jugadas
  max_playtime_user = playtime_by_user_year.groupby('user_id')['playtime_forever'].sum().idxmax()

  # Obtiene una lista de la acumulación de horas jugadas por año para el usuario con más horas jugadas
  playtime_by_year = playtime_by_user_year[(playtime_by_user_year['user_id'] == max_playtime_user) & (playtime_by_user_year['release_year'] != -1)][['release_year', 'playtime_forever']].rename(columns={'release_year': 'Año', 'playtime_forever': 'Horas'}).to_dict('records')

  response = {f"Usuario con más horas jugadas para el género {genre}" : max_playtime_user, "Horas jugadas": playtime_by_year}

  return response

In [6]:
UserForGenre('RPG')

{'Usuario con más horas jugadas para el género RPG': 'shinomegami',
 'Horas jugadas': [{'Año': 1999, 'Horas': 377},
  {'Año': 2000, 'Horas': 0},
  {'Año': 2001, 'Horas': 0},
  {'Año': 2002, 'Horas': 0},
  {'Año': 2003, 'Horas': 530943},
  {'Año': 2004, 'Horas': 696},
  {'Año': 2005, 'Horas': 185},
  {'Año': 2006, 'Horas': 2023},
  {'Año': 2007, 'Horas': 135020},
  {'Año': 2008, 'Horas': 1305},
  {'Año': 2009, 'Horas': 1382},
  {'Año': 2010, 'Horas': 41407},
  {'Año': 2011, 'Horas': 17752},
  {'Año': 2012, 'Horas': 12819},
  {'Año': 2013, 'Horas': 107775},
  {'Año': 2014, 'Horas': 81939},
  {'Año': 2015, 'Horas': 111419},
  {'Año': 2016, 'Horas': 15550},
  {'Año': 2017, 'Horas': 0}]}

---
+ def **UsersRecommend( *`año` : int* )**:
   Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
  
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [7]:
def UsersRecommend(year: int):
    """
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
    (reviews.recommend = True y comentarios positivos/neutrales)

    Args:
      year: El año a filtrar.

    Returns:
      El top 3 de juegos recomendados.
      Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
    """

    # Filtra el DataFrame por año y recomendación
    df_year = df_recommend[(df_recommend['posted_year'] == year) & (df_recommend['recommend'] == True) & (df_recommend['sentiment_analysis'] > 1)]

    # Agrupa por juego y cuenta el número de recomendaciones para cada juego
    recommendations = df_year.groupby('item_name').size()

    # Ordena los juegos en función del número de recomendaciones y toma el top 3
    top_games = recommendations.sort_values(ascending=False).head(3)

    response = [{"Puesto {}".format(i+1): game} for i, game in enumerate(top_games.index)]

    return response

In [8]:
UsersRecommend(2013)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': "Garry's Mod"},
 {'Puesto 3': 'Dota 2'}]

---
+ def **UsersNotRecommend( *`año` : int* )**:
   Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
  
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [9]:
def UsersNotRecommend(year: int):
    """
    Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

    Args:
      year: El año a filtrar.

    Returns:
      El top 3 de juegos no recomendados.
      Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
    """

    # Filtra el DataFrame por año y no recomendación
    df_year = df_recommend[(df_recommend['posted_year'] == year) & (df_recommend['recommend'] == False) & (df_recommend['sentiment_analysis'] < 1)]

    # Agrupa por juego y cuenta el número de no recomendaciones para cada juego
    not_recommendations = df_year.groupby('item_name').size()

    # Ordena los juegos en función del número de no recomendaciones y toma el top 3
    top_games = not_recommendations.sort_values(ascending=False).head(3)

    response = [{"Puesto {}".format(i+1): game} for i, game in enumerate(top_games.index)]

    return response

In [10]:
UsersNotRecommend(2013)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Call of Duty®: Ghosts'},
 {'Puesto 3': 'Ace of Spades: Battle Builder'}]

---
+ def **sentiment_analysis( *`año` : int* )**:
    Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [11]:
def sentiment_analysis(year: int):
    """
    Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

    Args:
      year: El año a filtrar.

    Returns:
      El análisis de sentimiento para el año dado.
      Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}
    """

    # Filtra el DataFrame por año
    df_year = df_sentiment[df_sentiment['release_year'] == year]

    # Cuenta el número de registros de reseñas que se encuentren categorizados con un análisis de sentimiento
    sentiment_counts = df_year['sentiment_analysis'].value_counts().rename({0: 'Negative', 1: 'Neutral', 2: 'Positive'})

    response = sentiment_counts.to_dict()

    return response

In [12]:
sentiment_analysis(2017)

{'Positive': 1156, 'Neutral': 493, 'Negative': 455}